## 모델

체크포인트를 바탕으로 모델을 인스턴스화할 때 편리한 `AutoModel` 클래스 사용

* `AutoModel` 클래스

다양한 모델에 대한 단순한 wrapper\
체크포인트에 적합한 모델 아키텍처를 자동으로 추측한 뒤 모델을 인스턴스화함

### 트랜스포머 모델 생성

BERT 모델을 초기화하기 위해 가장 먼저 configuration 객체를 로드해야 함

In [1]:
from transformers import BertConfig, BertModel

# config(설정)을 만들기
config = BertConfig()

# 해당 config에서 모델을 생성
model = BertModel(config)

In [2]:
# configuration 객체에는 모델을 빌드하는데 필요한 속성이 포함됨
print(config)

BertConfig {
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.30.2",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}



### 다른 로딩 메소드들

기본 설정에서 모델을 생성하면 해당 모델을 임의의 값으로 초기화

```python
from transformers import BertConfig, BertModel

config = BertConfig()
model = BertModel(config)

# 모델은 무작위로 초기화
```

사전 학습된 Transformer 모델을 로드하여 사용

In [3]:
from transformers import BertModel

model = BertModel.from_pretrained("bert-base-cased")

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


`BertModel` 을 동일한 기능을 수행하는 `AutoModel` 클래스로 대체 가능

위의 코드는 `BertConfig` 를 사용하지 않고 `bert-base-cased` 식별자를 통해 모델을 로드함\
해당 모델은 체크포인트의 모든 가중치로 초기화되고, 새로운 작업에 대해 미세 조정할 수 있다

자동으로 가중치가 다운로드되고 캐시되어 캐시 폴더에 저장


### 저장 메소드

config.json: 모델 아키텍처를 구축하는 데 필요한 다양한 속성들\
pytorch_model.bin: `state dictionay` 라고도 불림, 모델의 모든 가중치가 저장됨

설정 객체는 모델의 아키텍처를 파악하는데 필요\
모델 가중치는 모델의 매개변수

In [5]:
# from_pretrained()와 유사한 save_pretrained() 메서드를 사용
model.save_pretrained("saving_folder")

### 트랜스포머 모델을 활용한 추론

모델을 사용하여 몇 가지 예측 수행

예) 두 개의 시퀀스

```python
sequences = ["Hello!", "Cool.", "Nice!"]
```

토크나이저는 이를 input IDs 라고 하는 어휘 인덱스로 변환한다\
출력은 다음과 같다

```python
encoded_sequences = [
    [101, 7592, 999, 102],
    [101, 4658, 1012, 102],
    [101, 3835, 999, 102],
]
```

In [6]:
encoded_sequences = [
    [101, 7592, 999, 102],
    [101, 4658, 1012, 102],
    [101, 3835, 999, 102],
]

In [7]:
# 모델의 입력으로 텐서 활용
# 입력을 매개변수로 지정하여 모델을 호출하기

import torch

model_inputs = torch.tensor(encoded_sequences)
output = model(model_inputs)

In [8]:
output

BaseModelOutputWithPoolingAndCrossAttentions(last_hidden_state=tensor([[[ 4.4496e-01,  4.8276e-01,  2.7797e-01,  ..., -5.4032e-02,
           3.9393e-01, -9.4770e-02],
         [ 2.4943e-01, -4.4093e-01,  8.1772e-01,  ..., -3.1917e-01,
           2.2992e-01, -4.1172e-02],
         [ 1.3668e-01,  2.2518e-01,  1.4502e-01,  ..., -4.6915e-02,
           2.8224e-01,  7.5566e-02],
         [ 1.1789e+00,  1.6739e-01, -1.8187e-01,  ...,  2.4671e-01,
           1.0441e+00, -6.1966e-03]],

        [[ 3.6436e-01,  3.2464e-02,  2.0258e-01,  ...,  6.0111e-02,
           3.2451e-01, -2.0995e-02],
         [ 7.1866e-01, -4.8725e-01,  5.1740e-01,  ..., -4.4012e-01,
           1.4553e-01, -3.7545e-02],
         [ 3.3223e-01, -2.3271e-01,  9.4876e-02,  ..., -2.5268e-01,
           3.2172e-01,  8.1120e-04],
         [ 1.2523e+00,  3.5754e-01, -5.1320e-02,  ..., -3.7840e-01,
           1.0526e+00, -5.6255e-01]],

        [[ 2.4042e-01,  1.4718e-01,  1.2110e-01,  ...,  7.6062e-02,
           3.3564e-01,  2